In [1]:
mymodel <- lm(dist~speed,cars)
mymodel


Call:
lm(formula = dist ~ speed, data = cars)

Coefficients:
(Intercept)        speed  
    -17.579        3.932  


In [2]:
# require() = library()
newd <- data.frame(speed=c(10, 30, 50, 100))
newd

speed
<dbl>
10
30
50
100


In [3]:
predict(mymodel)

1         2         3         4         5         6         7         8 
-1.849460 -1.849460  9.947766  9.947766 13.880175 17.812584 21.744993 21.744993 
        9        10        11        12        13        14        15        16 
21.744993 25.677401 25.677401 29.609810 29.609810 29.609810 29.609810 33.542219 
       17        18        19        20        21        22        23        24 
33.542219 33.542219 33.542219 37.474628 37.474628 37.474628 37.474628 41.407036 
       25        26        27        28        29        30        31        32 
41.407036 41.407036 45.339445 45.339445 49.271854 49.271854 49.271854 53.204263 
       33        34        35        36        37        38        39        40 
53.204263 53.204263 53.204263 57.136672 57.136672 57.136672 61.069080 61.069080 
       41        42        43        44        45        46        47        48 
61.069080 61.069080 61.069080 68.933898 72.866307 76.798715 76.798715 76.798715 
       49        50 
76.798715 80.731124

## k-NN 알고리즘 (K-Nearest Neighbors / 유유상종)

In [4]:
wbcd <- read.csv("rexp/wisc_bc_data.csv", stringsAsFactors = TRUE) # 분류
# 특정컬럼 팩터 변경 > factor(wbcd$diagnosis, levels=c("B"."M"), levels = c("Benign","Malignent"))

In [5]:
str(wbcd)

'data.frame':	569 obs. of  32 variables:
 $ id               : int  87139402 8910251 905520 868871 9012568 906539 925291 87880 862989 89827 ...
 $ diagnosis        : Factor w/ 2 levels "B","M": 1 1 1 1 1 1 1 2 1 1 ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 

### 다양한 특성값이 주어져 있을때의 진단결과 분류

In [6]:
wbcd<-wbcd[-1] # -(n) 해당번호 제외 컬럼 출력 c(-1, -3, -5) 

In [7]:
summary(wbcd$diagnosis) #table(wbcd$diagnosis)

B   M 
357 212

In [8]:
typeof(table(wbcd$diagnosis))

[1] "integer"

In [9]:
table(wbcd$diagnosis)[1]

B 
357

In [10]:
round(prop.table(table(wbcd$diagnosis))*100,1) # proportion (%)


   B    M 
62.7 37.3 

In [11]:
summary(wbcd[c('radius_mean','area_mean', 'smoothness_mean')])
# 정규화 (서로다른 열을 정규화하여 오류없는 유클리디안 거리 구하기)
# 분자 분모에 뭔얘길 하시는데 뭔지 하나도 몰으겠음 >_<

  radius_mean       area_mean      smoothness_mean  
 Min.   : 6.981   Min.   : 143.5   Min.   :0.05263  
 1st Qu.:11.700   1st Qu.: 420.3   1st Qu.:0.08637  
 Median :13.370   Median : 551.1   Median :0.09587  
 Mean   :14.127   Mean   : 654.9   Mean   :0.09636  
 3rd Qu.:15.780   3rd Qu.: 782.7   3rd Qu.:0.10530  
 Max.   :28.110   Max.   :2501.0   Max.   :0.16340  

In [12]:
# 정규화작업함수
normalize <- function(x) {
    return ((x-min(x)) / (max(x) - min(x)))
}
print(normalize (c(1,2,3,4,5))) # 벡터 : 

[1] 0.00 0.25 0.50 0.75 1.00


In [13]:
# 데이터프레임 : 동일한 길이의 벡터들로 구성된 리스트
# lapply (List apply) : 리스트에 저장된 자료(열벡터)에 대해 특정함수를 적용
wbcd_n<-as.data.frame(lapply(wbcd[2:31],normalize))

In [14]:
wbcd_n

radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.2526859,0.0906324,0.2422777,0.13599152,0.4529205,0.15468376,0.093416120,0.18389662,0.4540404,0.20197978,...,0.1981501,0.09648188,0.1820808,0.08943669,0.4446279,0.09635106,0.099201278,0.32271478,0.2487680,0.08310376
0.1712812,0.3124789,0.1761454,0.08606575,0.3994764,0.29237470,0.149648547,0.13131213,0.4353535,0.31486942,...,0.1405194,0.29104478,0.1388017,0.05888714,0.3310440,0.21752966,0.153035144,0.27237113,0.2710428,0.13662600
0.1921056,0.2407846,0.1874784,0.09743372,0.4971563,0.17992761,0.071368322,0.12326044,0.3303030,0.28306655,...,0.1593739,0.38432836,0.1470193,0.07034015,0.4340619,0.11730749,0.085223642,0.25536082,0.2824759,0.15590975
0.2034644,0.1244505,0.2018520,0.10235419,0.5756974,0.28900067,0.108598875,0.23836978,0.3590909,0.22662174,...,0.1419424,0.09994670,0.1300862,0.06114825,0.4327412,0.15029446,0.069241214,0.29591065,0.1058545,0.08395645
0.3885182,0.1183632,0.3721927,0.24106045,0.2437483,0.15324213,0.079498594,0.13205765,0.3338384,0.11541702,...,0.2942014,0.09888060,0.2693859,0.15579532,0.2735918,0.14204771,0.108785942,0.28103093,0.1817465,0.08277581
0.2171896,0.3155225,0.2101444,0.11291622,0.2963799,0.17741243,0.128514527,0.07097416,0.4904040,0.26769166,...,0.1828531,0.39872068,0.1793914,0.08240759,0.3548174,0.16145181,0.204472843,0.22900344,0.2897694,0.18234291
0.2143499,0.4808928,0.2123557,0.11028632,0.3609280,0.25372677,0.260543580,0.20402584,0.1656566,0.33150800,...,0.1618641,0.67004264,0.1587230,0.07102831,0.3871756,0.21772371,0.289936102,0.33171821,0.1078257,0.21172767
0.3232051,0.4748055,0.3301085,0.19270414,0.7192381,0.48285381,0.365042174,0.45606362,0.6015152,0.51074136,...,0.4009249,0.79504264,0.3889138,0.23785883,1.0000000,0.47890289,0.371086262,0.69175258,0.5651488,0.35130526
0.1660751,0.3239770,0.1632230,0.08169671,0.4266498,0.20366849,0.070173383,0.05969185,0.5843434,0.31276327,...,0.1284240,0.30090618,0.1185816,0.05348014,0.3350063,0.11769557,0.063793930,0.11006873,0.2485709,0.13433032


In [15]:
print(summary(wbcd_n$area_mean))
print(range(wbcd$area_mean))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.1174  0.1729  0.2169  0.2711  1.0000 
[1]  143.5 2501.0


In [16]:
# 모델 훈련용 train set과 테스트용 test set으로 데이터셋 구분
# 일반적으로 전체 데이터셋의 70%를 training set, 30%를 test set으로 구분
# 트레이닝 데이터 > 알고리즘 > 모델링 > 모델 (분류 및 예측, 알고리즘) > 테스트 > 결과 > 모델 개선 (의미있는 정확도가 나올때까지 개선)

In [17]:
wbcd_train<-wbcd_n[1:400,]
wbcd_test<-wbcd_n[401:569,]

In [18]:
wbcd_train_labels<-wbcd[1:400,1] # 정답데이터 : Label
wbcd_test_labels<-wbcd[401:569,1]

In [19]:
library(class) #class::knn

In [20]:
wbcd_test_pred<-knn(train=wbcd_train, test=wbcd_test, cl=wbcd_train_labels, k=5)

In [21]:
print(table(wbcd_test_pred))
print(wbcd_test_pred)

wbcd_test_pred
  B   M 
109  60 
  [1] B B M M B M B B M B B B B B B M B B M M B M B B B M B B B M M M B M B B M
 [38] B B B B B M B M B B B B M B M M B B B B M B B M M B B M B B M B B B B B M
 [75] B M B M B M B B M B B M B M B M M M M B B B B M M M M M M B B B B B M M B
[112] M M B M M M M M B B B B B B B B M B B B B B M M B B B B B M B B M M B B B
[149] B B B B M B B M B B B B M B B B B B B B M
Levels: B M


In [22]:
# 교차분석 : 변수간 관련성의 여부 분석
library("gmodels")

In [23]:
CrossTable(x=wbcd_test_labels, y=wbcd_test_pred)
# N 자료의 갯수
# chai-square contribution  카이제곱
# N/row total 행의 갯수
# N/col total 열의 갯수
# N/table total


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  169 

 
                 | wbcd_test_pred 
wbcd_test_labels |         B |         M | Row Total | 
-----------------|-----------|-----------|-----------|
               B |       106 |         0 |       106 | 
                 |    20.715 |    37.633 |           | 
                 |     1.000 |     0.000 |     0.627 | 
                 |     0.972 |     0.000 |           | 
                 |     0.627 |     0.000 |           | 
-----------------|-----------|-----------|-----------|
               M |         3 |        60 |        63 | 
                 |    34.855 |    63.319 |           | 
                 |     0.048 |     0.952 |     0.373 | 
                 |     0.028 |     1.000 |           | 
                 |     0.018 |     0.3

In [24]:
x = c(10.6,18.95,69.28,346.4,0.09688,0.1147,0.06387,0.02642,0.1922,0.06491,0.4505,1.197,3.43,27.1,0.00747,0.03581,0.03354,0.01365,0.03504,0.003318,11.88,22.94,78.28,424.8,0.1213,0.2515,0.1916,0.07926,0.294,0.07587)

In [27]:
wbcd_test_ex <- knn(train=wbcd_train, test=x, cl=wbcd_train_labels, k=5)
wbcd_test_ex

[1] M
Levels: B M

In [28]:
# 표준화 : 정규화보단, (극단치)
# 거리 계산 : 평균으로부터 얼마만큼 떨어져 있는지 (가중치 적용이 가능함.)
# z-점수 표준화

In [30]:
head(scale(wbcd[-1]))

radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
-0.5128453,-1.60418301,-0.5399006,-0.5421468,0.4578825,-0.6538379,-0.6137661,-0.30717196,0.5376080,-0.45997776,...,-0.5729467,-1.6330626,-0.60385945,-0.5822061,0.2685394,-0.81141409,-0.70935407,-0.3148560,-0.11921566,-0.89893012
-1.0009202,-0.07896900,-0.9337442,-0.8766033,0.0369535,0.1961461,-0.3127117,-0.57983238,0.4026419,0.29919003,...,-0.9081255,-0.4453479,-0.86247083,-0.8005226,-0.4847751,-0.01757408,-0.38628525,-0.5377296,0.06343283,-0.44713458
-0.8760638,-0.57187353,-0.8662517,-0.8004484,0.8062867,-0.4980044,-0.7318045,-0.62158190,-0.3560868,0.08532000,...,-0.7984682,0.1241043,-0.81336741,-0.7186759,0.1984636,-0.67412871,-0.79323693,-0.6130350,0.15718162,-0.28435531
-0.8079604,-1.37168088,-0.7806514,-0.7674858,1.4248817,0.1753178,-0.5324814,-0.02471844,-0.1481659,-0.29426389,...,-0.8998495,-1.6119115,-0.91455023,-0.7843640,0.1897041,-0.45803135,-0.88915098,-0.4335191,-1.29107549,-0.89173240
0.3015589,-1.41353126,0.2337944,0.1617181,-1.1895712,-0.6627373,-0.6882771,-0.57596668,-0.3305526,-1.04210082,...,-0.0143154,-1.6184195,-0.08217273,-0.1079870,-0.8658121,-0.51205569,-0.65183440,-0.4993923,-0.66877751,-0.90169847
-0.7256686,-0.05804381,-0.7312666,-0.6967299,-0.7750414,-0.5135309,-0.4258580,-0.89269604,0.8002448,-0.01807412,...,-0.6619139,0.2119626,-0.61992966,-0.6324382,-0.3271047,-0.38493959,-0.07759635,-0.7297203,0.21698688,-0.06122589


In [35]:
wbcd_z <- as.data.frame(scale(wbcd[-1]))
summary(wbcd_z$area_mean)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.4532 -0.6666 -0.2949  0.0000  0.3632  5.2459 